# Lab 01: Baseline Agent

## Overview

In this notebook, we deploy an **intentionally unoptimized** customer support agent to establish baseline metrics. This baseline will be used to measure improvements in subsequent notebooks.

**What you'll learn:**
- How to deploy an agent to AgentCore Runtime
- How to configure Langfuse for observability
- How to invoke agents via the AgentCore API
- How to establish baseline metrics for cost and latency

## Prerequisites

- AWS account with Bedrock and AgentCore access
- Langfuse account (free tier works)
- `.env` file configured with your credentials

## Workshop Journey

```
[01 Baseline] → 02 Quick Wins → 03 Caching → 04 Routing → 05 Guardrails → 06 Gateway → 07 Evaluations
     ↑
  You are here
```

## Step 1: Setup and Dependencies

Before starting, ensure you've run `uv sync` in the terminal and selected the `.venv` kernel in VS Code.

In [1]:
from __future__ import annotations

import json
import os
import uuid
from pathlib import Path

from dotenv import load_dotenv

load_dotenv(override=True)

import boto3
from bedrock_agentcore_starter_toolkit import Runtime

region = os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
control_client = boto3.client("bedrock-agentcore-control", region_name=region)
data_client = boto3.client("bedrock-agentcore", region_name=region)
agentcore_runtime = Runtime()

print(f"Region: {region}")
print(f"Langfuse Host: {os.environ.get('LANGFUSE_BASE_URL', 'Not set')}")

Region: us-east-1
Langfuse Host: https://d2rhlwziq3nnbf.cloudfront.net


## Step 2: Review the Baseline Agent

Our baseline agent (`agents/v1_baseline.py`) is intentionally unoptimized:

- **Verbose system prompt** (~1500 tokens instead of ~250)
- **No max_tokens limit** (model can generate unlimited output)
- **No prompt caching** (system prompt processed every request)
- **No stop sequences** (model decides when to stop)

Let's examine the key parts of the baseline agent:

In [2]:
# Read and display the baseline agent code
agent_file = Path("agents/v1_baseline.py")
print(agent_file.read_text())

"""
V1 Baseline Agent - Intentionally unoptimized for comparison.
No caching, verbose prompt, no max_tokens limit.
"""

import base64
import os
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from dotenv import load_dotenv
from strands import Agent
from strands.models import BedrockModel
from strands.telemetry import StrandsTelemetry

import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from utils.tools import get_return_policy, get_product_info, web_search, get_technical_support

load_dotenv()

# Langfuse configuration
langfuse_public_key = os.environ.get("LANGFUSE_PUBLIC_KEY")
langfuse_secret_key = os.environ.get("LANGFUSE_SECRET_KEY")
langfuse_base_url = os.environ.get("LANGFUSE_BASE_URL", "https://cloud.langfuse.com")
LANGFUSE_AUTH = base64.b64encode(f"{langfuse_public_key}:{langfuse_secret_key}".encode()).decode()

os.environ["LANGFUSE_PROJECT_NAME"] = "my-llm-project"
os.environ["DISABLE_ADOT_OBSERVABILITY"] = "true"
os.environ["OTE

## Step 3: Configure the Agent for Deployment

Now we'll configure the agent for deployment to AgentCore Runtime.

In [3]:
agent_name = "customer_support_v1_baseline"
agent_file = str(Path("agents/v1_baseline.py").absolute())
requirements_file = str(Path("requirements-for-agentcore.txt").absolute())

print(f"Agent name: {agent_name}")
print(f"Agent file: {agent_file}")
print(f"Requirements: {requirements_file}")

Agent name: customer_support_v1_baseline
Agent file: /Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/agents/v1_baseline.py
Requirements: /Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/requirements-for-agentcore.txt


In [4]:
# Configure the runtime
print(f"Configuring agent: {agent_name}")
agentcore_runtime.configure(
    entrypoint=agent_file,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file=requirements_file,
    region=region,
    agent_name=agent_name,
)

Entrypoint parsed: file=/Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/agents/v1_baseline.py, bedrock_agentcore_name=v1_baseline
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: customer_support_v1_baseline


Configuring agent: customer_support_v1_baseline


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Memory disabled
Network mode: PUBLIC
Generated Dockerfile: Dockerfile
Generated .dockerignore: /Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/.dockerignore
Keeping 'customer_support_v1_baseline' as default agent
Bedrock AgentCore configured: /Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/Dockerfile'), dockerignore_path=PosixPath('/Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/.dockerignore'), runtime='None', runtime_type=None, region='us-east-1', account_id='739907928487', execution_role=None, ecr_repository=None, auto_create_ecr=True, s3_path=None, auto_create_s3=False, memory_id=None, network_mode='PUBLIC', network_subnets=None, network_security_groups=None, network_vpc_id=None)

## Step 4: Modify Dockerfile for Langfuse

We need to disable the default OpenTelemetry instrumentation and use Langfuse instead.

In [5]:
dockerfile_path = Path("Dockerfile")
if dockerfile_path.exists():
    content = dockerfile_path.read_text()
    # Replace opentelemetry-instrument wrapper with direct python call
    # Keep the correct module path: agents.v1_baseline
    if "opentelemetry-instrument" in content:
        import re
        content = re.sub(
            r'CMD \["opentelemetry-instrument", "python", "-m", "([^"]+)"\]',
            r'CMD ["python", "-m", "\1"]',
            content
        )
        dockerfile_path.write_text(content)
        print("Dockerfile modified for Langfuse")
    else:
        print("Dockerfile already configured or using different format")
else:
    print("Dockerfile not found - will be created during deployment")

Dockerfile modified for Langfuse


## Step 5: Deploy to AgentCore Runtime

Deploy the agent with Langfuse environment variables.

In [6]:
env_vars = {
    "LANGFUSE_BASE_URL": os.environ.get("LANGFUSE_BASE_URL"),
    "LANGFUSE_PUBLIC_KEY": os.environ.get("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.environ.get("LANGFUSE_SECRET_KEY"),
    "PYTHONUNBUFFERED": "1",
}

print("Deploying to AgentCore Runtime...")
print("This may take 5-10 minutes for the first deployment.")
launch_result = agentcore_runtime.launch(env_vars=env_vars, auto_update_on_conflict=True)
print(f"Agent deployed: {launch_result.agent_arn}")

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'customer_support_v1_baseline' to account 739907928487 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: customer_support_v1_baseline


Deploying to AgentCore Runtime...
This may take 5-10 minutes for the first deployment.


ECR repository available: 739907928487.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-customer_support_v1_baseline
Getting or creating execution role for agent: customer_support_v1_baseline
Using AWS region: us-east-1, account ID: 739907928487
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-d683549b2a


✅ Reusing existing ECR repository: 739907928487.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-customer_support_v1_baseline


✅ Reusing existing execution role: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-d683549b2a
Execution role available: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-d683549b2a
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: customer_support_v1_baseline
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-d683549b2a
Reusing existing CodeBuild execution role: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-d683549b2a
Using dockerignore.template with 46 patterns for zip filtering
Uploaded source to S3: customer_support_v1_baseline/source.zip
Updated CodeBuild project: bedrock-agentcore-customer_support_v1_baseline-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.3s
🔄 PROVISIONING started (total: 2s)
✅ PROVISIONING completed in 7.7s
🔄 DOWNLOAD

Agent deployed: arn:aws:bedrock-agentcore:us-east-1:739907928487:runtime/customer_support_v1_baseline-q4CFH7BNIH


In [7]:
# Save the agent ARN for later use
agent_arn = launch_result.agent_arn
print(f"Agent ARN: {agent_arn}")

Agent ARN: arn:aws:bedrock-agentcore:us-east-1:739907928487:runtime/customer_support_v1_baseline-q4CFH7BNIH


In [8]:

agent_arn='arn:aws:bedrock-agentcore:us-east-1:739907928487:runtime/customer_support_v1_baseline-q4CFH7BNIH'

## Step 6: Test the Baseline Agent

Let's invoke the agent with some test queries to establish our baseline metrics.

In [9]:
def invoke_agent(prompt):
    """Invoke the agent via AgentCore API."""
    response = data_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=json.dumps({"prompt": prompt}).encode(),
    )
    return json.loads(response["response"].read().decode("utf-8"))

In [10]:
# Import Langfuse metrics helper
from utils.langfuse_metrics import (
    clear_metrics,
    collect_metric,
    get_latest_trace_metrics,
    print_metrics,
    print_metrics_table,
)

# Clear any previously collected metrics
clear_metrics()

def invoke_agent_with_metrics(prompt, test_name=""):
    """Invoke the agent and fetch + print Langfuse metrics."""
    response = invoke_agent(prompt)
    print(response)
    
    # Fetch and print metrics from Langfuse
    metrics = get_latest_trace_metrics(
        agent_name="customer-support-v1-baseline",
        wait_seconds=5,
        max_retries=5,
        timeout_seconds=120,
    )
    print_metrics(metrics, test_name)
    
    # Collect metrics for summary table
    collect_metric(metrics, test_name)
    
    return response, metrics

In [11]:
# Standard test prompts - same across all notebooks for consistent comparison
TEST_PROMPTS = [
    # Single tool: get_return_policy
    ("Return Policy", "What is your return policy for laptops?"),

    # Single tool: get_product_info
    ("Product Info", "Tell me about your smartphone options"),

    # Single tool: get_technical_support (Bedrock KB)
    ("Technical Support", "My laptop won't turn on, can you help me troubleshoot?"),

    # Multi-tool: get_product_info + get_return_policy
    ("Multi-part Question", "I want to buy a laptop. What are the specs and what's the return policy?"),

    # No tool: General greeting
    ("General Question", "Hello! What can you help me with today?"),
]

# Run all tests and collect metrics
for test_name, prompt in TEST_PROMPTS:
    print("=" * 60)
    print(f"Test: {test_name}")
    print("=" * 60)
    response, metrics = invoke_agent_with_metrics(prompt, test_name=test_name)

Test: Return Policy
I'd be happy to help you with our laptop return policy!

Here are the details for returning laptops at TechMart Electronics:

**Return Window:** You have 30 days from the date of purchase to return your laptop.

**Condition Requirements:** The laptop must be in its original packaging with all accessories included and have no physical damage.

**How to Return:**
- Use our online RMA (Return Merchandise Authorization) portal, or
- Bring it to one of our stores for an in-store return

**Refund Timeline:** You'll receive your refund within 7-10 business days after we inspect the returned item.

**Return Shipping:**
- **Free** if the laptop is defective
- **Customer pays** for returns due to a change of mind

**Restocking Fee:**
- **No fee** if the laptop is defective
- **15% restocking fee** if you're returning it due to a change of mind

**Warranty:** All laptops come with a 1-year manufacturer warranty, and we also offer extended warranty options if you're interested.

In [12]:
# Print summary table of all test metrics
print_metrics_table()


                                  METRICS SUMMARY
               Test Latency    Cost Input Output Cache Read Tokens Cache Write Tokens
      Return Policy   8.12s $0.0209 5,347    326                 0                  0
       Product Info   9.24s $0.0229 5,426    439                 0                  0
  Technical Support  10.92s $0.0234 5,459    468                 0                  0
Multi-part Question  11.13s $0.0257 5,717    573                 0                  0
   General Question   5.11s $0.0112 2,574    234                 0                  0
---------------------------------------------------------------------------------------------------------
  TOTALS: Latency(avg): 8.90s | Cost: $0.1042 | Input: 24,523 | Output: 2,040
          Cache Read Tokens: 0 | Cache Write Tokens: 0



,Test,Latency,Cost,Input,Output,Cache Read Tokens,Cache Write Tokens
0,Return Policy,8.12s,$0.0209,"5,347",326,0,0
1,Product Info,9.24s,$0.0229,"5,426",439,0,0
2,Technical Support,10.92s,$0.0234,"5,459",468,0,0
3,Multi-part Question,11.13s,$0.0257,"5,717",573,0,0
4,General Question,5.11s,$0.0112,"2,574",234,0,0


## Step 7: View Langfuse Dashboard

Now let's check the Langfuse dashboard to see our baseline metrics.

### What to look for:

1. **Token Usage**: Note the input and output token counts
2. **Latency**: Time taken for each request
3. **Cost**: Estimated cost per request
4. **No Cache Hits**: `cacheReadInputTokens` should be 0 (no caching)

### Dashboard URL

In [13]:
langfuse_base_url = os.environ.get("LANGFUSE_BASE_URL", "https://cloud.langfuse.com")
print(f"View your traces at: {langfuse_base_url}")
print("\nFilter by tags: 'baseline', 'no-optimization'")
print("\nMetrics to record:")
print("- Average input tokens: _____")
print("- Average output tokens: _____")
print("- Average latency: _____ ms")
print("- Cache read tokens: 0 (expected)")

View your traces at: https://d2rhlwziq3nnbf.cloudfront.net

Filter by tags: 'baseline', 'no-optimization'

Metrics to record:
- Average input tokens: _____
- Average output tokens: _____
- Average latency: _____ ms
- Cache read tokens: 0 (expected)


## Summary

In this notebook, we:

1. Deployed an unoptimized baseline agent to AgentCore Runtime
2. Configured Langfuse for observability
3. Ran test scenarios to establish baseline metrics
4. Identified areas for optimization:
   - Verbose system prompt (~1500 tokens)
   - No output token limits
   - No prompt caching
   - No model routing

**Next Steps:** In the next notebook, we'll apply "quick wins" optimizations:
- Concise system prompt
- max_tokens limit
- stop_sequences

**Next notebook:** [02-quick-wins.ipynb](./02-quick-wins.ipynb)

## Cleanup (Optional)

Uncomment and run if you want to delete the agent.

In [14]:
# Uncomment to delete the agent
# agent_id = agent_arn.split("/")[-1]
# control_client.delete_agent_runtime(agentRuntimeId=agent_id)
# print(f"Deleted agent: {agent_name}")